https://requests-oauthlib.readthedocs.io/en/latest/oauth2_workflow.html#web-application-flow
https://hub.marvelstrikeforce.com/en/developer
https://developer.marvelstrikeforce.com/beta/index.html

In [ ]:
pip install --upgrade pip

In [ ]:
pip install requests-oauthlib

In [1]:
from requests_oauthlib import OAuth2Session
import pandas as pd
from time import time

In [2]:
from instance import config
client_id = config.MSF_API_CLIENT_ID
client_secret = config.MSF_API_CLIENT_SECRET

In [3]:
redirect_uri = 'https://localhost:2705/callback'
api_key = r'17wMKJLRxy3pYDCKG5ciP7VSU45OVumB2biCzzgw'
api_server = 'https://api.marvelstrikeforce.com'

In [4]:
scope = ['m3p.f.pr.pro' ,
         #'m3p.f.pr.ros',
         'm3p.f.pr.inv',
         'm3p.f.pr.act',
         'openid', 
         'offline'
         ]
oauth = OAuth2Session(client_id, redirect_uri=redirect_uri,
                          scope=scope)
oauth.headers.update({'x-api-key' : api_key})
authorization_url, state = oauth.authorization_url(
        'https://hydra-public.prod.m3.scopelypv.com/oauth2/auth')

print ('Please go to %s and authorize access.' % authorization_url)
authorization_response = input('Enter the full callback URL')

token = oauth.fetch_token(
        'https://hydra-public.prod.m3.scopelypv.com/oauth2/token',
        authorization_response=authorization_response, client_secret=client_secret)

endpoint = "/game/v1/languages"
params = {"page": 1,
          "perPage": 1}
r = oauth.get(api_server + endpoint, params=params)
ping = r.json()
current_chars_hash = ping["meta"]["hashes"]["chars"]
print("Connected, gear_hash: %s" % current_chars_hash)

Please go to https://hydra-public.prod.m3.scopelypv.com/oauth2/auth?response_type=code&client_id=0e31e793-86c3-40c6-9116-1e9e316299fe&redirect_uri=https%3A%2F%2Flocalhost%3A2705%2Fcallback&scope=m3p.f.pr.pro+m3p.f.pr.inv+m3p.f.pr.act+openid+offline&state=nvlmE6CDNAwbIyYbuqjbe914MSLdNF and authorize access.
Connected, gear_hash: 8ccbfe58465a800b55e9


In [5]:
r = oauth.get(api_server+"/player/v1/inventory")
r.json()
inventory=r.json()

In [ ]:
r = oauth.get(api_server+"/game/v1/characters", params={"charInfo":"none", "status" : "playable"})
characters = r.json()

In [6]:
characterId = "Punisher"
endpoint = "/game/v1/characters/" + characterId
params = {"statsFormat": "csv",
          "charInfo": "none",
          "costumes": "none",
          "abilityKits": "none",
          "pieceFlatCost": "full",
          "subPieceInfo": "none"}
r = oauth.get(api_server + endpoint, params=params)
punisher = r.json()

In [7]:
totalCost = {}
for tier in punisher["data"]["gearTiers"].values():
    if ("slots" in tier):
        for slot in tier["slots"]:
            if ("flatCost" in slot["piece"]):
                for item in slot["piece"]["flatCost"]:
                    #print("%s - %s" % (item["item"]["id"], item["quantity"]))
                    if (item["item"]["id"] in totalCost):
                        totalCost[item["item"]["id"]] += item["quantity"]
                    else:
                        totalCost[item["item"]["id"]] = item["quantity"]
            else:
                #print("%s - 1" % slot["piece"]["id"])
                if (slot["piece"]["id"] in totalCost):
                        totalCost[slot["piece"]["id"]] += 1
                else:
                    totalCost[slot["piece"]["id"]] = 1

In [27]:
def get_flat_cost (gear_id: str) -> dict:
    totalFlatCost = {}
    gear = get_gear(gear_id)['data']
    if ("flatCost" in gear):
        for item in gear["flatCost"]:
            if (item["item"]["id"] in totalFlatCost):
                totalFlatCost[item["item"]["id"]] += item["quantity"]
            else:
                totalFlatCost[item["item"]["id"]] = item["quantity"]
    return totalFlatCost

In [28]:
get_flat_cost("GEAR_T1_PUNISHER")

{'GEAR_GREEN_DAMAGE_MAT': 1, 'GEAR_GREEN_FOCUS_MAT': 1}

In [8]:
totalDirectCost = {}
for tier in punisher["data"]["gearTiers"].values():
    if ("slots" in tier):
        for slot in tier["slots"]:
            if ("flatCost" in slot["piece"]):
                if (slot["piece"]["id"] in totalDirectCost):
                        totalDirectCost[slot["piece"]["id"]] += 1
                else:
                    totalDirectCost[slot["piece"]["id"]] = 1

In [9]:
def get_direct_cost(gear: dict, quantity: int):
    if "directCost" in gear["item"]:
        if (gear["item"]["id"] in recursiveDirectCost):
            recursiveDirectCost[gear["item"]["id"]] += gear["quantity"] * quantity
        else:
            recursiveDirectCost[gear["item"]["id"]] = gear["quantity"] * quantity
        for piece in gear["item"]["directCost"]:
            get_direct_cost(piece, gear["quantity"])

In [100]:
recursiveDirectCost = {}
for (current_gear_id, quantity) in totalDirectCost.items():
    current_gear = get_gear(current_gear_id)
    for piece in current_gear["data"]["directCost"]:
        get_direct_cost(piece, quantity)

In [ ]:
totalCost

In [ ]:
totalDirectCost

In [ ]:
recursiveDirectCost

In [ ]:
json.dumps(inventory)

In [72]:
def find_in_inventory(cost: dict) -> list:
    costInv = []
    for currentItem in cost:
        currentCost = [inventoryItem["quantity"] for inventoryItem in inventory["data"] if currentItem == inventoryItem["item"]]
        if (len(currentCost) == 0):
            currentCost = [0]
        costInv.extend(currentCost)
    cost_array = pd.DataFrame(list(cost.items()))
    cost_array.columns = ["Name", "Needed"]
    cost_array.insert(2, "Inventory", costInv)
    return cost_array

In [81]:
totalCostInv = find_in_inventory(totalCost)
totalRecursiveDirectCostInv = find_in_inventory(totalDirectCost | recursiveDirectCost)

In [85]:
crafted_gear = totalRecursiveDirectCostInv.loc[totalRecursiveDirectCostInv["Inventory"] > 0]

In [ ]:
for index, gear in crafted_gear.iterrows():
    print(get_flat_cost(gear["Name"]))
    print(min(gear["Needed"], gear["Inventory"]))

In [84]:
print (totalCostInv.shape)


(66, 3)


In [4]:
import numpy as np
import pandas as pd
import csv

In [18]:
with open('some.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(totalCostInv)
    writer.writerow(['totalDirectCostInv'])
    writer.writerows(totalDirectCostInv)
    writer.writerow(["recursiveDirectCostInv"])
    writer.writerows(recursiveDirectCostInv)

https://rednafi.github.io/digressions/python/database/2020/05/25/python-redis-cache.html#getting-route-data-from-mapbox-api
redis-stack-server

In [ ]:
pip install redis

In [4]:
import json
import sys
from datetime import timedelta
import redis

In [10]:
def redis_connect() -> redis.client.Redis:
    try:
        client = redis.Redis(
            host="localhost",
            port=6379,
#            password="ubuntu",
            db=0,
            socket_timeout=5,
        )
        ping = client.ping()
        if ping is True:
            return client
    except redis.AuthenticationError:
        print("AuthenticationError")
        sys.exit(1)

redis_client = redis_connect()

def check_hash():
    return
    """cache_hash = redis_client.get("char_hash").decode()
    if cache_hash != current_chars_hash:
        print("Hash mismathch: %s %s", (cache_hash, current_chars_hash))
        redis_client.flushdb()
        redis_client.set("char_hash", current_chars_hash)"""
#check_hash()

def get_gear_from_api(gear_id: str) -> dict:
    """Data from api"""

    endpoint = "/game/v1/items/" + gear_id
    params = {"statsFormat": "csv",
            #  "pieceInfo": "none",
              "pieceDirectCost": "full",
              "pieceFlatCost": "full",
              "subPieceInfo": "none"}
    r = oauth.get(api_server + endpoint, params=params)
    data = r.json()
    #if (current_chars_hash != data["meta"]["hashes"]["chars"]):
    current_chars_hash = data["meta"]["hashes"]["chars"]
    check_hash()
    return data

def get_data_from_cache(key: str) -> str:
    """Data from redis."""
    
    val = redis_client.get(key)
    if val is not None:
        val = json.loads(val)
        val["cache"] = True
        return val
    else:
        return (val)

def set_data_to_cache(key: str, value: str) -> bool:
    """Data to redis."""

    state = redis_client.set(key, value=json.dumps(value))
    return state

def get_gear(gear: str) -> dict:

    # First it looks for the data in redis cache
    data = get_data_from_cache(key=gear)

    # If cache is found then serves the data from cache
    if data is not None:
        return data

    else:
        # If cache is not found then sends request to the MapBox API
        data = get_gear_from_api(gear)
        # This block sets saves the respose to redis and serves it directly
        data["cache"] = False
        state = set_data_to_cache(key=gear, value=data)
        return data

In [11]:
check_hash()

In [44]:
endpoint = "/player/v1/events"
params = {"statsFormat": "csv",
          "itemFormat": "id",
          "traitFormat": "id",
          "type": "milestone",
          "objRewards": "full",
          "pieceInfo": "none"}
r = oauth.get(api_server + endpoint, params=params)
events = r.json()
current_time = time()
current_gold_milestone = [event for event in events["data"] if event["name"] == "Golden Opportunity" and event["startTime"] < current_time and event["endTime"] > current_time][0]["milestone"]
current_gold_milestone_points = current_gold_milestone["brackets"][0]["objective"]["progress"]["points"]
current_gold_milestone_goal = current_gold_milestone["brackets"][0]["objective"]["tiers"][str(max([int(x) for x in current_gold_milestone["brackets"][0]["objective"]["tiers"].keys()]))]["goal"]
current_gold_milestone_delta = current_gold_milestone_goal - current_gold_milestone_points
current_gold_milestone_delta

5715000

In [7]:
def get_campaigns_from_api() -> dict:
    """Data from api"""

    endpoint = "/game/v1/episodics/" + "campaign"
    params = {"statsFormat": "csv",
          "itemFormat": "id",
          "traitFormat": "id"}
    r = oauth.get(api_server + endpoint, params=params)
    data = r.json()

    #if (current_chars_hash != data["meta"]["hashes"]["chars"]):
    current_nodes_hash = data["meta"]["hashes"]["nodes"]
    check_hash()
    return data

def get_campaigns() -> dict:

    # First it looks for the data in redis cache
    data = get_data_from_cache(key="nodes_campaigns")

    # If cache is found then serves the data from cache
    if data is not None:
        return data
    else:
        # If cache is not found then sends request to the MapBox API
        data = get_campaigns_from_api()
        # This block sets saves the respose to redis and serves it directly
        data["cache"] = False
        state = set_data_to_cache(key="nodes_campaign", value=data)
        return data

def get_campaign_names() -> dict:

    # First it looks for the data in redis cache
    data = get_data_from_cache(key="nodes_campaign_names")

    # If cache is found then serves the data from cache
    if data is not None:
        return data
    else:
        # If cache is not found then sends request to the MapBox API
        campaigns = get_campaigns()
        # This block sets saves the respose to redis and serves it directly
        campaign_ids = [current_campaign["id"] for current_campaign in campaigns["data"]]
        data = {"data": campaign_ids, "meta": campaigns["meta"]}
        data["cache"] = False
        state = set_data_to_cache(key="nodes_campaign_names", value=data)
        return data

In [8]:
def get_campaign_from_api(campaign_name: str) -> dict:
    """Data from api"""
    endpoint = "/game/v1/episodics/" + "campaign" + "/" + campaign_name
    params = {"statsFormat": "csv",
              "itemFormat": "id",
              "traitFormat": "id",
              "nodeInfo": "part",
              "nodeRewards": "full",
              "pieceInfo": "none"}
    r = oauth.get(api_server + endpoint, params=params)
    data = r.json()

    #if (current_chars_hash != data["meta"]["hashes"]["chars"]):
    current_nodes_hash = data["meta"]["hashes"]["nodes"]
    check_hash()
    return data

def get_campaign(campaign_name: str) -> dict:

    # First it looks for the data in redis cache
    data = get_data_from_cache(key="nodes_" + campaign_name)

    # If cache is found then serves the data from cache
    if data is not None:
        return data
    else:
        # If cache is not found then sends request to the MapBox API
        data = get_campaign_from_api(campaign_name)
        # This block sets saves the respose to redis and serves it directly
        data["cache"] = False
        state = set_data_to_cache(key="nodes_" + campaign_name, value=data)
        return data

In [12]:
get_campaign_names()["data"]

['VILLAINS_CAMPAIGN',
 'NEXUS_CAMPAIGN',
 'COSMIC_CAMPAIGN',
 'MYSTIC_CAMPAIGN',
 'HEROES_CAMPAIGN',
 'DOOM_CAMPAIGN',
 'ISO8_CAMPAIGN']

In [17]:
all_rewards_by_id = {}
for current_campaign_name in get_campaign_names()["data"]:
    current_campaign = get_campaign(current_campaign_name)
    for current_chapter_num, current_chapter_data in current_campaign["data"]["chapters"].items():
        for current_tier_num, current_tier_data in current_chapter_data["tiers"].items():
            for current_reward_group in current_tier_data["rewards"]["allOf"]:
                for current_reward_group_key, current_reward_group_value in current_reward_group.items():
                    match current_reward_group_key:
                        case "item":
                            if (current_reward_group_value != "SC" and current_reward_group_value != "XP"):
                                #print (current_reward_group_value)
                                reward = [{"campaign_id": current_campaign_name, "chapter": current_chapter_num, "tier": current_tier_num}]
                                if (current_reward_group_value not in all_rewards_by_id):
                                    all_rewards_by_id[current_reward_group_value] = reward
                                else:
                                    all_rewards_by_id[current_reward_group_value].append(reward)
                            ""
                        case "chanceOf":
                            for current_reward_group_key_chahnceof, current_reward_group_value_chanceof in current_reward_group_value.items():
                                match current_reward_group_key_chahnceof:
                                    case "item":
                                        if (current_reward_group_value_chanceof != "SC" and current_reward_group_value_chanceof != "XP"):
                                            #print (current_reward_group_value_chanceof)
                                            reward = [{"campaign_id": current_campaign_name, "chapter": current_chapter_num, "tier": current_tier_num}]
                                            if (current_reward_group_value_chanceof not in all_rewards_by_id):
                                                all_rewards_by_id[current_reward_group_value_chanceof] = reward
                                            else:
                                                all_rewards_by_id[current_reward_group_value_chanceof].append(reward)
                                        ""
                                    case "quantity":
                                        ""
                                    case _:
                                        print (current_reward_group_key_chahnceof)
                        case "quantity":
                            ""
                        case _:
                            print (current_reward_group_key)
                        

In [18]:
with open('all_rewards_by_id.json', 'w') as f:
    json.dump(all_rewards_by_id, f)

In [ ]:
json.dumps(list(all_rewards_by_id.keys()))